In [1]:
import pandas as pd

In [2]:
#read csv file with pandas
data = pd.read_csv('data/stunting21.csv')
data.drop(columns=['No', 'NIK', 'Nama', 'NIK', 'Tgl Lahir', 'BB Lahir', 'TB Lahir', 'Nama Ortu', 'Prov', 'Kab/Kota', 'LiLA', 'Jml Vit A', 'Kec', 'Pukesmas', 'Desa/Kel', 'Posyandu', 'Alamat', 'Tanggal Pengukuran', 'Naik Berat Badan', 'BB/U', 'TB/U', 'BB/TB'], inplace = True)
data.head()

,JK,Usia Saat Ukur,Berat,Tinggi,ZS BB/U,ZS TB/U,ZS BB/TB
0,L,2 Tahun - 11 Bulan - 16 Hari,10:05:00 AM,86:05:00,-2.56,-2.52,-1.73
1,L,1 Tahun - 6 Bulan - 11 Hari,08:03:00 AM,79,-2.53,-1.33,-2.65
2,P,3 Tahun - 10 Bulan - 8 Hari,12:01:00 PM,93:04:00,-2.01,-1.96,-1.24
3,L,3 Tahun - 8 Bulan - 1 Hari,12:04:00 PM,93,-1.9,-1.99,-1.16
4,L,3 Tahun - 10 Bulan - 2 Hari,06:04:00 PM,101,01:08:00 AM,-0.3,0.104861111


In [3]:
#fix invalid values in data
def convert_weight_height(value):
    try:
        if isinstance(value, str):
            if ':' in value and 'AM' in value or 'PM' in value:  # Check for time format
                parts = value.split(':')
                if len(parts) == 3:
                    hours = int(parts[0])
                    minutes = int(parts[1])
                    seconds = int(parts[2].split(' ')[0])
                    am_pm = parts[2].split(' ')[1]
                    if am_pm == "PM" and hours != 12:
                        hours += 12
                    elif am_pm == "AM" and hours == 12:
                        hours = 0
                    return hours + minutes / 60 + seconds / 3600
            elif ':' in value:  # Check for non-time format like "XX:YY:ZZ"
                parts = value.split(':')
                if len(parts) == 3:
                    return int(parts[0]) + int(parts[1])/60 + int(parts[2])/3600
            elif value.isdigit():  # Handle simple integer string format
                return float(value)
        return float(value)  # For already numeric values
    except Exception:
        return None

def convert_age_to_months(age):
    if isinstance(age, str):
        parts = age.split(' - ')
        years = int(parts[0].split(' ')[0])
        months = int(parts[1].split(' ')[0])
        days = int(parts[2].split(' ')[0])
        total_months = years * 12 + months + days / 30.44  # Approximate month length
        return round(total_months, 2)
    return age 

data["Berat"] = data["Berat"].apply(convert_weight_height)
data["Tinggi"] = data["Tinggi"].apply(convert_weight_height)
data["ZS BB/U"] = data["ZS BB/U"].apply(convert_weight_height)
data["ZS BB/TB"] = data["ZS BB/TB"].apply(convert_weight_height)
data['ZS TB/U'] = data['ZS TB/U'].apply(convert_weight_height)
data["Usia Saat Ukur"] = data["Usia Saat Ukur"].apply(convert_age_to_months)

In [4]:
#check for collumns with null data
data.isnull().sum()

JK                23
Usia Saat Ukur    23
Berat             23
Tinggi            23
ZS BB/U           23
ZS TB/U           23
ZS BB/TB          23
dtype: int64

In [5]:
#delete row with null data
data.dropna(inplace=True)
print(data.isnull().sum())

JK                0
Usia Saat Ukur    0
Berat             0
Tinggi            0
ZS BB/U           0
ZS TB/U           0
ZS BB/TB          0
dtype: int64


In [6]:
#rename the collumn to make it simple
data.rename(columns={"JK": "gender", "Usia Saat Ukur": "age", "Berat": "weight", "Tinggi": "height", "ZS BB/U": "zs_weight_age", "ZS TB/U": "zs_height_age", "ZS BB/TB": "zs_weight_height"}, inplace=True)

In [7]:
#replace gender labels
data['gender'] = data['gender'].replace({'L': 'male', 'P': 'female'}, regex=True)
data['gender'] = data['gender'].apply(lambda x: x.strip())

In [8]:
data.head()

,gender,age,weight,height,zs_weight_age,zs_height_age,zs_weight_height
0,male,35.53,10.083333,86.083333,-2.560000,-2.52,-1.730000
1,male,18.36,8.050000,79.000000,-2.530000,-1.33,-2.650000
2,female,46.26,12.016667,93.066667,-2.010000,-1.96,-1.240000
3,male,44.03,12.066667,93.000000,-1.900000,-1.99,-1.160000
4,male,46.07,18.066667,101.000000,1.133333,-0.30,0.104861


In [9]:
#round the age data to integer
data['age'] = data['age'].astype(int)
#round all collumn to .2f
columns_to_convert = ['weight', 'height', 'zs_weight_age', 'zs_height_age', 'zs_weight_height']
for col in columns_to_convert:
    converted_values = data[col].apply(convert_weight_height)
    data[col] = converted_values.round(2) if converted_values.dtype != 'O' else converted_values

In [10]:
#new collumn by sum the three zs values, then divide by 3 (mean for zsvalues)
data['totalzs_3'] = (data['zs_weight_age'] + data['zs_height_age'] + data['zs_weight_height']) / 3
data.head()

,gender,age,weight,height,zs_weight_age,zs_height_age,zs_weight_height,totalzs_3
0,male,35,10.08,86.08,-2.56,-2.52,-1.73,-2.270000
1,male,18,8.05,79.00,-2.53,-1.33,-2.65,-2.170000
2,female,46,12.02,93.07,-2.01,-1.96,-1.24,-1.736667
3,male,44,12.07,93.00,-1.90,-1.99,-1.16,-1.683333
4,male,46,18.07,101.00,1.13,-0.30,0.10,0.310000


In [11]:
#transform the totalzs/3 to percentage
#-3 to 0 is 100% to 0%
#0 to +3 is 0% to 100%
def transform_percentage(x):
    if -3 <= x < -0.00001:
        return (100 - (x / 3 * 100)) -100
    if 0.000001 <= x <= 3:
        return (x / 3 * 100)
data['totalzs_percentage'] = data['totalzs_3'].apply(transform_percentage)

columns_to_convert = ['totalzs_percentage']
for col in columns_to_convert:
    converted_values = data[col].apply(convert_weight_height)
    data[col] = converted_values.round(2) if converted_values.dtype != 'O' else converted_values

In [12]:
data.head(20)

,gender,age,weight,height,zs_weight_age,zs_height_age,zs_weight_height,totalzs_3,totalzs_percentage
0,male,35,10.08,86.08,-2.56,-2.52,-1.73,-2.270000,75.67
1,male,18,8.05,79.00,-2.53,-1.33,-2.65,-2.170000,72.33
2,female,46,12.02,93.07,-2.01,-1.96,-1.24,-1.736667,57.89
3,male,44,12.07,93.00,-1.90,-1.99,-1.16,-1.683333,56.11
4,male,46,18.07,101.00,1.13,-0.30,0.10,0.310000,10.33
5,male,15,9.08,74.05,-0.76,-1.95,0.30,-0.803333,26.78
6,female,43,10.10,92.00,-2.88,-1.97,-2.51,-2.453333,81.78
7,male,52,13.10,102.00,-1.79,-0.90,-1.95,-1.546667,51.56
8,female,36,11.10,88.00,-1.46,-1.92,-0.52,-1.300000,43.33
9,male,18,9.02,78.00,-1.81,-1.90,-1.26,-1.656667,55.22


In [13]:
data.to_csv('data/cleaned_data.csv', index=False)